Add all neccesary packages

In [228]:
from bs4 import BeautifulSoup
import requests
import os, os.path, csv
import pandas as pd
listingurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(listingurl)
soup = BeautifulSoup(response.text, "html.parser")
import re

Use beautiful soup to pull out wikipedia page and extract all the lines containing td.

This extracts all lines containing all information we want and some useless information, which is after line 287.

The useless information is removed by subsetting the first 286 entries of dataframe.

After that, all 'not assigned' borough entries have been removed

In [21]:
listings = []
n=0
c=0
df = pd.DataFrame()
for rows in soup.find_all("td"):
    n=n+1
    r=n%3
   # print(rows)
   # if ("oddrow" in rows["class"]) or ("evenrow" in rows["class"]):
       # name = rows.find("div", class_="name").a.get_text()
       # hometown = rows.find_all("td")[1].get_text()
       # school = hometown[hometown.find(",")+4:]
       # city = hometown[:hometown.find(",")+4]
    if (r==1):
     position = str(rows).index('</td>')
     content=str(rows)[4:position]
    
     #print(content)
     
     df.loc[c,0]=content
     c=c+1
    if (r==2):
     position = str(rows).index('</td>')
     content=str(rows)[4:position]
     if (len(content)>50): 
       position = str(rows).index('">')
       position2 = str(rows).index('</a>')
       content=str(rows)[position+2:position2]
     #print(content)
     
     df.loc[c-1,1]=content
    if (r==0):
     position = str(rows).index('</td>')
     content=str(rows)[4:position]
     #print(content)

     if (len(content)>50): 
         position = str(rows).index('">')
         position2 = str(rows).index('</a>')
         content=str(rows)[position+2:position2]
         position3=[]
     df.loc[c-1,2]=content 

#remove extra rows    
df=df.loc[:287,]    
#remove /n
n=-1
for rows in df.loc[:,2]:
    n=n+1
    try:
       position=str(rows).index('\n')
       df.loc[n,2]=str(rows)[:position]
       
    except:
       a=1  
#rename the columns
df.rename(columns={0:'PostalCode',
                      1:    'Borough',
                       2:   'Neighborhood'}, 
                 inplace=True)
#remove not assigned
df=df[df.Borough != 'Not assigned']
#df=df[df.Neighborhood != 'Not assigned']
df = df.reset_index()
df=df.drop(['index'],axis=1)
#df2=pd.unique(df.PostalCode)

 There are some postal codes containing more than one neighborhood. In this step, these neighborhood are combined into a single entry of the postal code

In [222]:
##concatenate Neighborhood
df3=pd.unique(df.PostalCode)
n=-1
for pc in df3:
    n=n+1
    if n==0:
        if len(df[df.PostalCode==df3[n]])==1:
          df2=df[df.PostalCode==df3[n]]
        else:
         
          df2=pd.concat([df2,df[df.PostalCode==df3[n]]])
    else:
         if len(df[df.PostalCode==df3[n]])==1:
                df2=pd.concat([df2,df[df.PostalCode==df3[n]]])
         else:       
            str0=''
            temp=df[df.PostalCode==df3[n]]
            temp=temp.reset_index() 
            temp= temp.drop(['index'],axis=1)
            
            for i in range(0,len(df[df.PostalCode==df3[n]])):

             if i==0:
                temp1=temp.loc[i,]
                str0=str0+temp1.Neighborhood
             else:
                temp1=temp.loc[i,]
                str0=str0+','+temp1.Neighborhood 
            transposed=pd.DataFrame(data=temp.iloc[0])
               
            df2=pd.concat([df2,transposed.T ])
            df2=df2.reset_index() 
            df2= df2.drop(['index'],axis=1)

            df2.loc[n,'Neighborhood']=str0

It turns out there is one entry has 'not assigned' neighborhood. It is replace by the Borough name.

In [223]:
df2[df2.Neighborhood=="Not assigned"]

,PostalCode,Borough,Neighborhood
4,M7A,Queen's Park,Not assigned


In [224]:
##add empty Neighborhood
df2.loc[4,'Neighborhood']="Queen's Park"

The following code shows the dimensions of the dataframe

In [226]:
df2.shape

(103, 3)

In [227]:
df2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"
